# CLEANING DATA

Authenticates to Space-Track, pulls GP_History for chosen satellites.

Cleans & validates types, de-dupes, sorts.

Derives semi-major axis, perigee/apogee altitudes from mean motion & eccentricity.

Builds daily series, rolling z-scores, and simple anomaly flags.

Exports a clean features table and an anomaly events CSV.

# RETRIEVING DATA

In [1]:
# run in a terminal cell once (or use requirements.txt)
!pip install requests python-dotenv pandas numpy sgp4 pandera ruptures matplotlib
!pip install --upgrade "pandas>=2.1.1" "pandera>=0.20.0"

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB 5.4 MB/s eta 0:00:03
   - -------------------------------------- 0.5/15.8 MB 6.7 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/15.8 MB 6.6 MB/s eta 0:00:03
   -- ------------------------------------- 1.1/15.8 MB 6.2 MB/s eta 0:00:03
   --- ------------------------------------ 1.3/15.8 MB 6.0 MB/s eta 0:00:03
   ---- ----------------------------------- 1.7/15.8 MB 6.3 MB/s eta 0:00:03
   ---- -----------------------

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\kruti\\anaconda3\\Lib\\site-packages\\~umpy.libs\\libscipy_openblas64_-860d95b1c38e637ce4509f5fa24fbf2a.dll'
Consider using the `--user` option or check the permissions.



In [2]:
!pip install --upgrade numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/d5/03/0eade211c504bda872a594f045f98ddcc6caef2b7c63610946845e304d3f/numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl (13.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.3.2 which is incompatible.
scipy 1.11.1 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.3.2 which is incompatible.


In [3]:
import sys, numpy as np
print("Kernel exe:", sys.executable)
print("NumPy:", np.__version__)

Kernel exe: C:\Users\kruti\anaconda3\python.exe
NumPy: 2.3.2


In [4]:
# This ensures pip targets the same interpreter used by Jupyter
!{sys.executable} -m pip install --upgrade "numpy>=1.24.4" "pandas>=2.1.1" "pandera>=0.20.0"

In [11]:
import requests
import pandas as pd

# Space-Track login
USERNAME = "krutiray95@gmail.com"
PASSWORD = "Kxrwebster9439316894!"

session = requests.Session()
login_url = "https://www.space-track.org/ajaxauth/login"
login_payload = {"identity": USERNAME, "password": PASSWORD}
session.post(login_url, data=login_payload)

# Query SATCAT CSV
url = "https://www.space-track.org/basicspacedata/query/class/satcat/orderby/NORAD_CAT_ID asc/format/csv"
response = session.get(url)

# Save CSV locally
with open("satcat.csv", "wb") as f:
    f.write(response.content)

# Load into pandas
df = pd.read_csv("satcat.csv")
print(df.head())
print(len(df), "rows loaded")

     INTLDES  NORAD_CAT_ID  OBJECT_TYPE     SATNAME COUNTRY      LAUNCH  \
0  1957-001A             1  ROCKET BODY    SL-1 R/B     CIS  1957-10-04   
1  1957-001B             2      PAYLOAD   SPUTNIK 1     CIS  1957-10-04   
2  1957-002A             3      PAYLOAD   SPUTNIK 2     CIS  1957-11-03   
3  1958-001A             4      PAYLOAD  EXPLORER 1      US  1958-02-01   
4  1958-002B             5      PAYLOAD  VANGUARD 1      US  1958-03-17   

    SITE       DECAY  PERIOD  INCLINATION  ...  RCSVALUE  RCS_SIZE  FILE  \
0  TTMTR  1957-12-01   96.19        65.10  ...         0     LARGE     1   
1  TTMTR  1958-01-03   96.10        65.00  ...         0       NaN  7179   
2  TTMTR  1958-04-14  103.74        65.33  ...         0     SMALL  9221   
3  AFETR  1970-03-31   88.48        33.15  ...         0       NaN     1   
4  AFETR         NaN  132.61        34.26  ...         0     SMALL  9242   

   LAUNCH_YEAR  LAUNCH_NUM LAUNCH_PIECE  CURRENT  OBJECT_NAME  OBJECT_ID  \
0         1957  

In [14]:
import os, requests, pandas as pd

USER = os.getenv("SPACETRACK_USER", "krutiray95@gmail.com")
PASS = os.getenv("SPACETRACK_PASS", "Kxrwebster9439316894!")

s = requests.Session()
s.post("https://www.space-track.org/ajaxauth/login",
       data={"identity": USER, "password": PASS})

url = ("https://www.space-track.org/basicspacedata/query/"
       "class/decay/DECAY_EPOCH/%3E2020-01-01/"
       "orderby/DECAY_EPOCH asc/format/csv")

r = s.get(url); r.raise_for_status()
open("decay.csv","wb").write(r.content)

df = pd.read_csv("decay.csv")
print(df.head()); print(len(df), "rows loaded")


   NORAD_CAT_ID  OBJECT_NUMBER     OBJECT_NAME     INTLDES   OBJECT_ID  RCS  \
0         44179          44179  MICROSAT-R DEB  2019-006BS  2019-006BS    0   
1         44179          44179  MICROSAT-R DEB  2019-006BS  2019-006BS    0   
2         26476          26476           DM-F3   2000-048A   2000-048A    0   
3         26476          26476           DM-F3   2000-048A   2000-048A    0   
4         44117          44117  MICROSAT-R DEB   2019-006C   2019-006C    0   

  RCS_SIZE COUNTRY            MSG_EPOCH         DECAY_EPOCH     SOURCE  \
0    SMALL     IND  2020-12-24 21:50:16  2020-01-01 0:00:00     satcat   
1    SMALL     IND  2020-01-06 20:44:00  2020-01-01 0:00:00  decay_msg   
2    LARGE      US  2019-12-29 09:02:00  2020-01-01 0:59:00    TIP_msg   
3    LARGE      US  2019-12-30 03:58:00  2020-01-01 4:08:00    TIP_msg   
4    SMALL     IND  2019-11-21 19:40:04  2020-01-02 0:00:00  60day_msg   

     MSG_TYPE  PRECEDENCE  
0  Historical           1  
1  Historical           

In [15]:
url = ("https://www.space-track.org/basicspacedata/query/"
       "class/decay/DECAY_EPOCH/2022-01-01--2025-01-01/"
       "orderby/DECAY_EPOCH asc/format/csv")

In [16]:
url = ("https://www.space-track.org/basicspacedata/query/"
       "class/satcat/DECAY/%3E2020-01-01/"
       "orderby/DECAY asc/format/csv")

# DATA CLEANING PROCESS